In [7]:
import streamlit as st
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuración de la página
st.set_page_config(
    page_title="Análisis de Portafolios - Demo Concurso",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Título principal
st.title("📊 Analizador de Portafolios de Inversión")
st.markdown("---")

# Sidebar - Entradas del usuario
st.sidebar.header("🎯 Configuración del Portafolio")

# Selección de empresas
st.sidebar.subheader("1. Selección de Empresas")
tickers_input = st.sidebar.text_area(
    "Símbolos (separados por coma):",
    value="AAPL, MSFT, GOOGL, TSLA, AMZN",
    help="Ingresa los símbolos de Yahoo Finance. Ejemplo: AAPL, MSFT, TSLA"
)

tickers = [ticker.strip().upper() for ticker in tickers_input.split(',') if ticker.strip()]

# Pesos del portafolio
st.sidebar.subheader("2. Asignación de Pesos")
st.sidebar.info(f"Empresas seleccionadas: {len(tickers)}")

weights = []
if len(tickers) > 0:
    for i, ticker in enumerate(tickers):
        weight = st.sidebar.slider(
            f"Peso {ticker}",
            min_value=0.0,
            max_value=1.0,
            value=1.0/len(tickers),
            step=0.01,
            help=f"Porcentaje asignado a {ticker}"
        )
        weights.append(weight)

    # Verificar suma de pesos
    total_weight = sum(weights)
    if abs(total_weight - 1.0) > 0.01:
        st.sidebar.error(f"⚠️ La suma de pesos es {total_weight:.2f}. Debe ser 1.0")
    else:
        st.sidebar.success(f"✅ Suma de pesos: {total_weight:.2f}")

# Parámetros de inversión
st.sidebar.subheader("3. Parámetros de Inversión")

initial_investment = st.sidebar.number_input(
    "Inversión Inicial ($)",
    min_value=1000,
    max_value=1000000,
    value=10000,
    step=1000
)

# Fechas de análisis
col1, col2 = st.sidebar.columns(2)
with col1:
    start_date = st.date_input(
        "Fecha Inicial",
        value=datetime.now() - timedelta(days=365*3)
    )
with col2:
    end_date = st.date_input(
        "Fecha Final",
        value=datetime.now()
    )

# Frecuencia temporal
frequency = st.sidebar.selectbox(
    "Frecuencia de Datos",
    options=["1d", "1wk", "1mo"],
    index=0,
    help="Frecuencia para cálculo de retornos"
)

# Benchmark
benchmark_ticker = st.sidebar.text_input(
    "Benchmark (opcional):",
    value="SPY",
    help="Símbolo para comparación (ej: SPY para S&P500)"
)

# Función para descargar datos
@st.cache_data(ttl=3600)
def download_data(tickers, start_date, end_date, frequency):
    try:
        data = yf.download(tickers, start=start_date, end=end_date, interval=frequency)
        if data.empty:
            return None
        return data
    except:
        return None

# Función para calcular métricas
def calculate_metrics(returns, weights, initial_investment):
    """Calcula todas las métricas del portafolio"""

    # Retornos del portafolio
    portfolio_returns = (returns * weights).sum(axis=1)

    # Retorno acumulado
    cumulative_returns = (1 + portfolio_returns).cumprod() - 1

    # Valor monetario del portafolio
    portfolio_value = initial_investment * (1 + cumulative_returns)

    # Métricas estadísticas
    total_return = cumulative_returns.iloc[-1]
    annual_return = portfolio_returns.mean() * 252
    volatility = portfolio_returns.std() * np.sqrt(252)
    sharpe_ratio = annual_return / volatility if volatility > 0 else 0

    # Máxima caída (Drawdown)
    cumulative = (1 + portfolio_returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min()

    return {
        'portfolio_returns': portfolio_returns,
        'cumulative_returns': cumulative_returns,
        'portfolio_value': portfolio_value,
        'total_return': total_return,
        'annual_return': annual_return,
        'volatility': volatility,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'drawdown': drawdown
    }

# Botón para ejecutar análisis
if st.sidebar.button("🚀 Ejecutar Análisis", type="primary"):
    with st.spinner("Descargando datos y calculando métricas..."):
        # Descargar datos
        data = download_data(tickers, start_date, end_date, frequency)

        if data is not None and len(tickers) > 0 and abs(sum(weights) - 1.0) < 0.01:

            # Obtener precios ajustados
            if 'Adj Close' in data.columns:
                prices = data['Adj Close']
            else:
                prices = data['Close']

            prices = prices.dropna()

            if not prices.empty:
                # Calcular retornos
                returns = prices.pct_change().dropna()

                # Calcular métricas del portafolio
                metrics = calculate_metrics(returns, weights, initial_investment)

                # Descargar benchmark si se especificó
                benchmark_data = None
                if benchmark_ticker.strip():
                    try:
                        bench_data = yf.download(benchmark_ticker, start=start_date, end=end_date, interval=frequency)
                        if 'Adj Close' in bench_data.columns:
                            bench_prices = bench_data['Adj Close']
                            bench_returns = bench_prices.pct_change().dropna()
                            bench_cumulative = (1 + bench_returns).cumprod() - 1
                            benchmark_data = {
                                'returns': bench_returns,
                                'cumulative': bench_cumulative,
                                'value': initial_investment * (1 + bench_cumulative)
                            }
                    except:
                        pass

                # Mostrar resultados
                st.success("✅ Análisis completado exitosamente!")

                # ===== SECCIÓN 1: MÉTRICAS PRINCIPALES =====
                st.header("📈 Métricas del Portafolio")

                # Tarjetas de métricas
                col1, col2, col3, col4 = st.columns(4)

                with col1:
                    st.metric(
                        "Retorno Total",
                        f"{metrics['total_return']:.2%}",
                        help="Retorno acumulado durante todo el período"
                    )

                with col2:
                    st.metric(
                        "Retorno Anualizado",
                        f"{metrics['annual_return']:.2%}",
                        help="Retorno anualizado (base 252 días)"
                    )

                with col3:
                    st.metric(
                        "Volatilidad Anual",
                        f"{metrics['volatility']:.2%}",
                        help="Desviación estándar anualizada de retornos"
                    )

                with col4:
                    st.metric(
                        "Ratio de Sharpe",
                        f"{metrics['sharpe_ratio']:.2f}",
                        help="Retorno por unidad de riesgo"
                    )

                # ===== SECCIÓN 2: VISUALIZACIONES =====
                st.header("📊 Visualizaciones")

                # Pestañas para diferentes gráficos
                tab1, tab2, tab3, tab4, tab5 = st.tabs([
                    "💰 Valor del Portafolio",
                    "📈 Retornos Acumulados",
                    "📉 Drawdown",
                    "🔍 Retornos Diarios",
                    "📊 Composición"
                ])

                with tab1:
                    # Evolución del valor monetario
                    fig, ax = plt.subplots(figsize=(12, 6))
                    ax.plot(metrics['portfolio_value'].index, metrics['portfolio_value'].values,
                           linewidth=2, label='Portafolio')

                    if benchmark_data is not None:
                        ax.plot(benchmark_data['value'].index, benchmark_data['value'].values,
                               linewidth=2, label=f'Benchmark ({benchmark_ticker})', alpha=0.7)

                    ax.set_title('Evolución del Valor del Portafolio', fontsize=16, fontweight='bold')
                    ax.set_xlabel('Fecha')
                    ax.set_ylabel('Valor ($)')
                    ax.legend()
                    ax.grid(True, alpha=0.3)
                    ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    st.pyplot(fig)

                with tab2:
                    # Retornos acumulados
                    fig, ax = plt.subplots(figsize=(12, 6))
                    ax.plot(metrics['cumulative_returns'].index, metrics['cumulative_returns'].values * 100,
                           linewidth=2, label='Portafolio')

                    if benchmark_data is not None:
                        ax.plot(benchmark_data['cumulative'].index, benchmark_data['cumulative'].values * 100,
                               linewidth=2, label=f'Benchmark ({benchmark_ticker})', alpha=0.7)

                    ax.set_title('Retornos Acumulados', fontsize=16, fontweight='bold')
                    ax.set_xlabel('Fecha')
                    ax.set_ylabel('Retorno Acumulado (%)')
                    ax.legend()
                    ax.grid(True, alpha=0.3)
                    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    st.pyplot(fig)

                with tab3:
                    # Drawdown
                    fig, ax = plt.subplots(figsize=(12, 6))
                    ax.fill_between(metrics['drawdown'].index, metrics['drawdown'].values * 100, 0,
                                  alpha=0.3, color='red')
                    ax.plot(metrics['drawdown'].index, metrics['drawdown'].values * 100,
                           color='red', linewidth=1)
                    ax.set_title('Drawdown del Portafolio', fontsize=16, fontweight='bold')
                    ax.set_xlabel('Fecha')
                    ax.set_ylabel('Drawdown (%)')
                    ax.grid(True, alpha=0.3)
                    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    st.pyplot(fig)

                with tab4:
                    # Histograma de retornos
                    fig, ax = plt.subplots(figsize=(12, 6))
                    ax.hist(metrics['portfolio_returns'].values * 100, bins=50, alpha=0.7,
                           edgecolor='black', color='skyblue')
                    ax.set_title('Distribución de Retornos Diarios', fontsize=16, fontweight='bold')
                    ax.set_xlabel('Retorno Diario (%)')
                    ax.set_ylabel('Frecuencia')
                    ax.grid(True, alpha=0.3)
                    plt.tight_layout()
                    st.pyplot(fig)

                with tab5:
                    # Composición del portafolio
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

                    # Pie chart de composición
                    ax1.pie(weights, labels=tickers, autopct='%1.1f%%', startangle=90)
                    ax1.set_title('Composición del Portafolio', fontweight='bold')

                    # Gráfico de barras de pesos
                    y_pos = np.arange(len(tickers))
                    ax2.barh(y_pos, weights, color='lightgreen')
                    ax2.set_yticks(y_pos)
                    ax2.set_yticklabels(tickers)
                    ax2.set_xlabel('Peso')
                    ax2.set_title('Asignación por Activo', fontweight='bold')
                    ax2.grid(True, alpha=0.3, axis='x')

                    plt.tight_layout()
                    st.pyplot(fig)

                # ===== SECCIÓN 3: DATOS DETALLADOS =====
                st.header("📋 Datos Detallados")

                # Retornos por período
                st.subheader("Retornos por Período")
                returns_df = pd.DataFrame({
                    'Retorno Diario': metrics['portfolio_returns'],
                    'Retorno Acumulado': metrics['cumulative_returns'],
                    'Valor Portafolio': metrics['portfolio_value']
                })
                st.dataframe(returns_df.tail(10), use_container_width=True)

                # Estadísticas adicionales
                st.subheader("Estadísticas Adicionales")
                col1, col2 = st.columns(2)

                with col1:
                    additional_stats = {
                        'Máximo Drawdown': f"{metrics['max_drawdown']:.2%}",
                        'Retorno Promedio Diario': f"{metrics['portfolio_returns'].mean():.4%}",
                        'Volatilidad Diaria': f"{metrics['portfolio_returns'].std():.4%}",
                        'Mejor Día': f"{metrics['portfolio_returns'].max():.2%}"
                    }

                    for key, value in additional_stats.items():
                        st.metric(key, value)

                with col2:
                    more_stats = {
                        'Peor Día': f"{metrics['portfolio_returns'].min():.2%}",
                        'Días Positivos': f"{(metrics['portfolio_returns'] > 0).sum()}",
                        'Días Negativos': f"{(metrics['portfolio_returns'] < 0).sum()}",
                        'Ratio Positivo/Negativo': f"{(metrics['portfolio_returns'] > 0).sum() / (metrics['portfolio_returns'] < 0).sum():.2f}"
                    }

                    for key, value in more_stats.items():
                        st.metric(key, value)

                # ===== SECCIÓN 4: DESCARGA DE DATOS =====
                st.header("📥 Exportar Resultados")

                # Preparar datos para descarga
                export_df = pd.DataFrame({
                    'Fecha': returns_df.index,
                    'Retorno_Diario': returns_df['Retorno Diario'],
                    'Retorno_Acumulado': returns_df['Retorno Acumulado'],
                    'Valor_Portafolio': returns_df['Valor Portafolio']
                })

                csv_data = export_df.to_csv(index=False)

                st.download_button(
                    label="📊 Descargar Datos del Análisis (CSV)",
                    data=csv_data,
                    file_name=f"analisis_portafolio_{datetime.now().strftime('%Y%m%d_%H%M')}.csv",
                    mime="text/csv"
                )

            else:
                st.error("No se pudieron obtener datos válidos para el análisis.")
        else:
            st.error("Verifica la configuración: símbolos válidos y pesos que sumen 1.0")

# Información inicial
else:
    st.markdown("""
    ## 🎯 Bienvenido al Analizador de Portafolios

    Esta aplicación te permite analizar y optimizar portafolios de inversión utilizando datos históricos de Yahoo Finance.

    ### 📋 Características principales:

    - **Selección flexible** de empresas y asignación de pesos
    - **Cálculo de métricas** profesionales: Sharpe, volatilidad, drawdown
    - **Visualizaciones interactivas** de rendimiento y riesgo
    - **Comparación con benchmark** (SPY, etc.)
    - **Exportación de resultados** para análisis posterior

    ### 🚀 Para comenzar:

    1. **Configura** tu portafolio en la barra lateral
    2. **Asigna** los pesos (deben sumar 1.0)
    3. **Define** la inversión inicial y período
    4. **Haz clic** en 'Ejecutar Análisis'

    ### 📊 Métricas que calculará:

    - Retornos periódicos y acumulados
    - Volatilidad histórica y anualizada
    - Ratio de Sharpe
    - Evolución del valor monetario
    - Máximo drawdown
    - Comparación con benchmark
    """)

# Footer
st.markdown("---")
st.caption("🎓 Demo para Concurso Universitario - Análisis Cuantitativo de Portafolios")

2025-11-06 04:57:05.293 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 04:57:05.296 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 04:57:05.300 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 04:57:05.304 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 04:57:05.306 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 04:57:05.311 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 04:57:05.312 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-06 04:57:05.320 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [11]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuración de la página
st.set_page_config(
    page_title="Análisis de Portafolios - Demo Concurso",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Título principal
st.title("📊 Analizador de Portafolios de Inversión")
st.markdown("---")

# Sidebar - Entradas del usuario
st.sidebar.header("🎯 Configuración del Portafolio")

# Selección de empresas
st.sidebar.subheader("1. Selección de Empresas")
tickers_input = st.sidebar.text_area(
    "Símbolos (separados por coma):",
    value="AAPL, MSFT, GOOGL, TSLA, AMZN",
    help="Ingresa los símbolos de Yahoo Finance. Ejemplo: AAPL, MSFT, TSLA"
)

tickers = [ticker.strip().upper() for ticker in tickers_input.split(',') if ticker.strip()]

# Pesos del portafolio
st.sidebar.subheader("2. Asignación de Pesos")
st.sidebar.info(f"Empresas seleccionadas: {len(tickers)}")

weights = []
if len(tickers) > 0:
    for i, ticker in enumerate(tickers):
        weight = st.sidebar.slider(
            f"Peso {ticker}",
            min_value=0.0,
            max_value=1.0,
            value=1.0/len(tickers),
            step=0.01,
            help=f"Porcentaje asignado a {ticker}"
        )
        weights.append(weight)

    # Verificar suma de pesos
    total_weight = sum(weights)
    if abs(total_weight - 1.0) > 0.01:
        st.sidebar.error(f"⚠️ La suma de pesos es {total_weight:.2f}. Debe ser 1.0")
    else:
        st.sidebar.success(f"✅ Suma de pesos: {total_weight:.2f}")

# Parámetros de inversión
st.sidebar.subheader("3. Parámetros de Inversión")

initial_investment = st.sidebar.number_input(
    "Inversión Inicial ($)",
    min_value=1000,
    max_value=1000000,
    value=10000,
    step=1000
)

# Fechas de análisis
col1, col2 = st.sidebar.columns(2)
with col1:
    start_date = st.date_input(
        "Fecha Inicial",
        value=datetime.now() - timedelta(days=365*3)
    )
with col2:
    end_date = st.date_input(
        "Fecha Final",
        value=datetime.now()
    )

# Frecuencia temporal
frequency = st.sidebar.selectbox(
    "Frecuencia de Datos",
    options=["1d", "1wk", "1mo"],
    index=0,
    help="Frecuencia para cálculo de retornos"
)

# Benchmark
benchmark_ticker = st.sidebar.text_input(
    "Benchmark (opcional):",
    value="SPY",
    help="Símbolo para comparación (ej: SPY para S&P500)"
)

# Función para descargar datos
@st.cache_data(ttl=3600)
def download_data(tickers, start_date, end_date, frequency):
    try:
        data = yf.download(tickers, start=start_date, end=end_date, interval=frequency)
        if data.empty:
            return None
        return data
    except:
        return None

# Función para calcular métricas
def calculate_metrics(returns, weights, initial_investment):
    """Calcula todas las métricas del portafolio"""

    # Retornos del portafolio
    portfolio_returns = (returns * weights).sum(axis=1)

    # Retorno acumulado
    cumulative_returns = (1 + portfolio_returns).cumprod() - 1

    # Valor monetario del portafolio
    portfolio_value = initial_investment * (1 + cumulative_returns)

    # Métricas estadísticas
    total_return = cumulative_returns.iloc[-1]
    annual_return = portfolio_returns.mean() * 252
    volatility = portfolio_returns.std() * np.sqrt(252)
    sharpe_ratio = annual_return / volatility if volatility > 0 else 0

    # Máxima caída (Drawdown)
    cumulative = (1 + portfolio_returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min()

    return {
        'portfolio_returns': portfolio_returns,
        'cumulative_returns': cumulative_returns,
        'portfolio_value': portfolio_value,
        'total_return': total_return,
        'annual_return': annual_return,
        'volatility': volatility,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'drawdown': drawdown
    }

# Botón para ejecutar análisis
if st.sidebar.button("🚀 Ejecutar Análisis", type="primary"):
    with st.spinner("Descargando datos y calculando métricas..."):
        # Descargar datos
        data = download_data(tickers, start_date, end_date, frequency)

        if data is not None and len(tickers) > 0 and abs(sum(weights) - 1.0) < 0.01:

            # Obtener precios ajustados
            if 'Adj Close' in data.columns:
                prices = data['Adj Close']
            else:
                prices = data['Close']

            prices = prices.dropna()

            if not prices.empty:
                # Calcular retornos
                returns = prices.pct_change().dropna()

                # Calcular métricas del portafolio
                metrics = calculate_metrics(returns, weights, initial_investment)

                # Descargar benchmark si se especificó
                benchmark_data = None
                if benchmark_ticker.strip():
                    try:
                        bench_data = yf.download(benchmark_ticker, start=start_date, end=end_date, interval=frequency)
                        if 'Adj Close' in bench_data.columns:
                            bench_prices = bench_data['Adj Close']
                            bench_returns = bench_prices.pct_change().dropna()
                            bench_cumulative = (1 + bench_returns).cumprod() - 1
                            benchmark_data = {
                                'returns': bench_returns,
                                'cumulative': bench_cumulative,
                                'value': initial_investment * (1 + bench_cumulative)
                            }
                    except:
                        pass

                # Mostrar resultados
                st.success("✅ Análisis completado exitosamente!")

                # ===== SECCIÓN 1: MÉTRICAS PRINCIPALES =====
                st.header("📈 Métricas del Portafolio")

                # Tarjetas de métricas
                col1, col2, col3, col4 = st.columns(4)

                with col1:
                    st.metric(
                        "Retorno Total",
                        f"{metrics['total_return']:.2%}",
                        help="Retorno acumulado durante todo el período"
                    )

                with col2:
                    st.metric(
                        "Retorno Anualizado",
                        f"{metrics['annual_return']:.2%}",
                        help="Retorno anualizado (base 252 días)"
                    )

                with col3:
                    st.metric(
                        "Volatilidad Anual",
                        f"{metrics['volatility']:.2%}",
                        help="Desviación estándar anualizada de retornos"
                    )

                with col4:
                    st.metric(
                        "Ratio de Sharpe",
                        f"{metrics['sharpe_ratio']:.2f}",
                        help="Retorno por unidad de riesgo"
                    )

                # ===== SECCIÓN 2: VISUALIZACIONES =====
                st.header("📊 Visualizaciones")

                # Pestañas para diferentes gráficos
                tab1, tab2, tab3, tab4, tab5 = st.tabs([
                    "💰 Valor del Portafolio",
                    "📈 Retornos Acumulados",
                    "📉 Drawdown",
                    "🔍 Retornos Diarios",
                    "📊 Composición"
                ])

                with tab1:
                    # Evolución del valor monetario
                    fig, ax = plt.subplots(figsize=(12, 6))
                    ax.plot(metrics['portfolio_value'].index, metrics['portfolio_value'].values,
                           linewidth=2, label='Portafolio')

                    if benchmark_data is not None:
                        ax.plot(benchmark_data['value'].index, benchmark_data['value'].values,
                               linewidth=2, label=f'Benchmark ({benchmark_ticker})', alpha=0.7)

                    ax.set_title('Evolución del Valor del Portafolio', fontsize=16, fontweight='bold')
                    ax.set_xlabel('Fecha')
                    ax.set_ylabel('Valor ($)')
                    ax.legend()
                    ax.grid(True, alpha=0.3)
                    ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    st.pyplot(fig)

                with tab2:
                    # Retornos acumulados
                    fig, ax = plt.subplots(figsize=(12, 6))
                    ax.plot(metrics['cumulative_returns'].index, metrics['cumulative_returns'].values * 100,
                           linewidth=2, label='Portafolio')

                    if benchmark_data is not None:
                        ax.plot(benchmark_data['cumulative'].index, benchmark_data['cumulative'].values * 100,
                               linewidth=2, label=f'Benchmark ({benchmark_ticker})', alpha=0.7)

                    ax.set_title('Retornos Acumulados', fontsize=16, fontweight='bold')
                    ax.set_xlabel('Fecha')
                    ax.set_ylabel('Retorno Acumulado (%)')
                    ax.legend()
                    ax.grid(True, alpha=0.3)
                    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    st.pyplot(fig)

                with tab3:
                    # Drawdown
                    fig, ax = plt.subplots(figsize=(12, 6))
                    ax.fill_between(metrics['drawdown'].index, metrics['drawdown'].values * 100, 0,
                                  alpha=0.3, color='red')
                    ax.plot(metrics['drawdown'].index, metrics['drawdown'].values * 100,
                           color='red', linewidth=1)
                    ax.set_title('Drawdown del Portafolio', fontsize=16, fontweight='bold')
                    ax.set_xlabel('Fecha')
                    ax.set_ylabel('Drawdown (%)')
                    ax.grid(True, alpha=0.3)
                    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    st.pyplot(fig)

                with tab4:
                    # Histograma de retornos
                    fig, ax = plt.subplots(figsize=(12, 6))
                    ax.hist(metrics['portfolio_returns'].values * 100, bins=50, alpha=0.7,
                           edgecolor='black', color='skyblue')
                    ax.set_title('Distribución de Retornos Diarios', fontsize=16, fontweight='bold')
                    ax.set_xlabel('Retorno Diario (%)')
                    ax.set_ylabel('Frecuencia')
                    ax.grid(True, alpha=0.3)
                    plt.tight_layout()
                    st.pyplot(fig)

                with tab5:
                    # Composición del portafolio
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

                    # Pie chart de composición
                    ax1.pie(weights, labels=tickers, autopct='%1.1f%%', startangle=90)
                    ax1.set_title('Composición del Portafolio', fontweight='bold')

                    # Gráfico de barras de pesos
                    y_pos = np.arange(len(tickers))
                    ax2.barh(y_pos, weights, color='lightgreen')
                    ax2.set_yticks(y_pos)
                    ax2.set_yticklabels(tickers)
                    ax2.set_xlabel('Peso')
                    ax2.set_title('Asignación por Activo', fontweight='bold')
                    ax2.grid(True, alpha=0.3, axis='x')

                    plt.tight_layout()
                    st.pyplot(fig)

                # ===== SECCIÓN 3: DATOS DETALLADOS =====
                st.header("📋 Datos Detallados")

                # Retornos por período
                st.subheader("Retornos por Período")
                returns_df = pd.DataFrame({
                    'Retorno Diario': metrics['portfolio_returns'],
                    'Retorno Acumulado': metrics['cumulative_returns'],
                    'Valor Portafolio': metrics['portfolio_value']
                })
                st.dataframe(returns_df.tail(10), use_container_width=True)

                # Estadísticas adicionales
                st.subheader("Estadísticas Adicionales")
                col1, col2 = st.columns(2)

                with col1:
                    additional_stats = {
                        'Máximo Drawdown': f"{metrics['max_drawdown']:.2%}",
                        'Retorno Promedio Diario': f"{metrics['portfolio_returns'].mean():.4%}",
                        'Volatilidad Diaria': f"{metrics['portfolio_returns'].std():.4%}",
                        'Mejor Día': f"{metrics['portfolio_returns'].max():.2%}"
                    }

                    for key, value in additional_stats.items():
                        st.metric(key, value)

                with col2:
                    more_stats = {
                        'Peor Día': f"{metrics['portfolio_returns'].min():.2%}",
                        'Días Positivos': f"{(metrics['portfolio_returns'] > 0).sum()}",
                        'Días Negativos': f"{(metrics['portfolio_returns'] < 0).sum()}",
                        'Ratio Positivo/Negativo': f"{(metrics['portfolio_returns'] > 0).sum() / (metrics['portfolio_returns'] < 0).sum():.2f}"
                    }

                    for key, value in more_stats.items():
                        st.metric(key, value)

                # ===== SECCIÓN 4: DESCARGA DE DATOS =====
                st.header("📥 Exportar Resultados")

                # Preparar datos para descarga
                export_df = pd.DataFrame({
                    'Fecha': returns_df.index,
                    'Retorno_Diario': returns_df['Retorno Diario'],
                    'Retorno_Acumulado': returns_df['Retorno Acumulado'],
                    'Valor_Portafolio': returns_df['Valor Portafolio']
                })

                csv_data = export_df.to_csv(index=False)

                st.download_button(
                    label="📊 Descargar Datos del Análisis (CSV)",
                    data=csv_data,
                    file_name=f"analisis_portafolio_{datetime.now().strftime('%Y%m%d_%H%M')}.csv",
                    mime="text/csv"
                )

            else:
                st.error("No se pudieron obtener datos válidos para el análisis.")
        else:
            st.error("Verifica la configuración: símbolos válidos y pesos que sumen 1.0")

# Información inicial
else:
    st.markdown("""
    ## 🎯 Bienvenido al Analizador de Portafolios

    Esta aplicación te permite analizar y optimizar portafolios de inversión utilizando datos históricos de Yahoo Finance.

    ### 📋 Características principales:

    - **Selección flexible** de empresas y asignación de pesos
    - **Cálculo de métricas** profesionales: Sharpe, volatilidad, drawdown
    - **Visualizaciones interactivas** de rendimiento y riesgo
    - **Comparación con benchmark** (SPY, etc.)
    - **Exportación de resultados** para análisis posterior

    ### 🚀 Para comenzar:

    1. **Configura** tu portafolio en la barra lateral
    2. **Asigna** los pesos (deben sumar 1.0)
    3. **Define** la inversión inicial y período
    4. **Haz clic** en 'Ejecutar Análisis'

    ### 📊 Métricas que calculará:

    - Retornos periódicos y acumulados
    - Volatilidad histórica y anualizada
    - Ratio de Sharpe
    - Evolución del valor monetario
    - Máximo drawdown
    - Comparación con benchmark
    """)

# Footer
st.markdown("---")
st.caption("🎓 Demo para Concurso Universitario - Análisis Cuantitativo de Portafolios")
!streamlit run app.py


Overwriting app.py
